In [307]:
import numpy as np
# import netCDF4 as nc
import xarray as xr
import pandas as pd

data_dir = '/mnt/lts/nfs_fs02/sadow_lab/personal/yusukemh/PI-CASC'
from config import FILE_NAMES, DF_LABELS

# enable autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [369]:
# process skn location data
df_locations = pd.read_csv(f'{data_dir}/SKNlocations.csv')
df_locations.columns = map(str.lower, df_locations.columns)
df_locations = df_locations.rename(columns={'lat_dd': 'lat', 'lon_dd': 'lon'})

# convert the longitude from (-180, 180) to (0, 360)
df_locations['lon_updated'] = df_locations['lon'] + 360
# for each row, compute the coordinates of the closest girdpoint in netCDF files
def closest_grid(lat, lon):
    # to save computation, only include grid points around Hawaii
    lat_grids = np.arange(12.5, 27.5, 2.5)
    lon_grids = np.arange(200, 212.5, 2.5)
    
    xx, yy = np.meshgrid(lat_grids, lon_grids)
    d = np.sqrt((xx-lat)**2 + (yy-lon)**2)
    target_idx = np.where(d == d.min())
    
    return tuple(np.array([xx[target_idx], yy[target_idx]]).reshape(-1,))

df_locations['closest_grid'] = df_locations.apply(lambda row: closest_grid(row['lat'], row['lon_updated']), axis=1)

In [371]:
df_locations.head()

,skn,name,lat,lon,lon_updated,closest_grid
0,1.0,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"
1,1.1,MORSE FIELD,18.913680,-155.680550,204.319450,"(20.0, 205.0)"
2,1.2,KALAE S TRK STA,18.938669,-155.680549,204.319451,"(20.0, 205.0)"
3,2.0,MANUKA,19.108660,-155.825545,204.174455,"(20.0, 205.0)"
4,2.1,KAHUKU MAUKA 2.10,19.108890,-155.746670,204.253330,"(20.0, 205.0)"


In [264]:
# process rainfall data
df_data = pd.read_excel(f"{data_dir}/FilledDataset2012.xlsx", sheet_name='Data_in')
df_data.columns = map(str.lower, df_data.columns)
# melt cells into rows
df_data = df_data.melt(
    id_vars=['skn', 'year'],
    var_name='month',
    value_name='data_in'
)

# convert string month to digit
month_to_digit = dict(
    jan=1, feb=2, mar=3, apr=4, may=5, jun=6,
    jul=7, aug=8, sep=9, oct=10, nov=11, dec=12
)
df_data.month = df_data.month.map(month_to_digit)
# replace empty space to NaN so the downstream process can remove those values
df_data['data_in'] = df_data['data_in'].replace(r'^\s*$', np.nan, regex=True)

In [265]:
df_data.head()

,skn,year,month,data_in
0,1.0,1920,1,4.76
1,1.0,1921,1,7.78
2,1.0,1922,1,4.02
3,1.0,1923,1,11.47
4,1.0,1924,1,0


In [261]:
# we need information on how each data is collected.
df_source = pd.read_excel(f"{data_dir}/FilledDataset2012.xlsx", sheet_name='Source')
df_source.columns = map(str.lower, df_source.columns)

# melt cells into rows
df_source = df_source.melt(
    id_vars=['skn', 'year'],
    var_name='month',
    value_name='filled'
)

b_filled = {
    # filled
    'Fill_1': True, 'Fill_2': True, 'Fill_3': True, 'Fill_4': True, 'Fill_5': True, 'NRFill': True,
    # nonfilled
    'State/NCDC': False, 'NCDC': False, 'State': False, 'Hydronet': False, 'RAWS': False,
    'SCAN': False, 'USGS': False, 'Hydronet/NCDC': False, 'HaleNet': False, 'HC&S': False,
    'AlanMair': False, 'USGS/State': False, 'USGS/NCDC': False, 'AlanMair/State': False,
}
df_source.filled = df_source.filled.map(b_filled)
df_source.month = df_source.month.map(month_to_digit)

In [262]:
df_source.head()

,skn,year,month,filled
0,1.0,1920,1,True
1,1.0,1921,1,True
2,1.0,1922,1,True
3,1.0,1923,1,True
4,1.0,1924,1,True


In [266]:
assert df_source.shape == df_data.shape

In [294]:
# merge df_data and df_source and drop rows with any NaN
df_data_w_source = pd.merge(left=df_data, right=df_source, left_on=['skn', 'year', 'month'], right_on=['skn', 'year', 'month']).dropna()

In [295]:
df_data_w_source.head()

,skn,year,month,data_in,filled
0,1.0,1920,1,4.76,True
1,1.0,1921,1,7.78,True
2,1.0,1922,1,4.02,True
3,1.0,1923,1,11.47,True
4,1.0,1924,1,0,True


In [372]:
# merge df_data_w_source and df_locations
df_ready_for_cdf = pd.merge(left=df_data_w_source, right=df_locations, left_on='skn', right_on='skn')
df_ready_for_cdf.head()

,skn,year,month,data_in,filled,name,lat,lon,lon_updated,closest_grid
0,1.0,1920,1,4.76,True,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"
1,1.0,1921,1,7.78,True,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"
2,1.0,1922,1,4.02,True,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"
3,1.0,1923,1,11.47,True,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"
4,1.0,1924,1,0,True,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)"


In [379]:
unique_closest_coords = np.array([np.array(item) for item in df_locations['closest_grid'].unique()])
unique_lat = np.unique(unique_closest_coords[:, 0])
unique_lon = np.unique(unique_closest_coords[:, 1])

In [414]:
dfs = []
for filename, df_label in zip(FILE_NAMES, DF_LABELS):
    df = xr.open_dataset(f"{data_dir}/reanalysis_data/{filename}").loc[dict(lat=unique_lat, lon=unique_lon)].to_dataframe() # only load relevant portion
    if "level" in df.index.names: # if there is extra level then we have to drop the level
        df = df.droplevel(level="level")
    df = df.reorder_levels(['lat', 'lon', 'time'])
    df.columns = [df_label]
    dfs.append(df)

In [415]:
df_cdf_combined = pd.concat(dfs, axis=1).reset_index()

In [417]:
df_cdf_combined['year'] = df_cdf_combined['time'].dt.year
df_cdf_combined['month'] = df_cdf_combined['time'].dt.month
df_cdf_combined['grid'] = df_cdf_combined.apply(lambda row: (row['lat'], row['lon']), axis=1)

In [421]:
df_w_closest_obs = pd.merge(
    left=df_ready_for_cdf,
    right=df_cdf_combined.drop(columns=['lat', 'lon']),
    left_on=['year', 'month', 'closest_grid'],
    right_on=['year', 'month', 'grid']
)

In [422]:
df_w_closest_obs.head()

,skn,year,month,data_in,filled,name,lat,lon,lon_updated,closest_grid,...,pr_wtr,shum-uwnd-700,shum-uwnd-925,shum-vwnd-700,shum-vwnd-950,shum700,shum925,skt,slp,grid
0,1.00,1948,1,3.2,False,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)",...,29.034512,2.592494,-25.859348,0.589191,7.106411,2.945999,9.869999,23.385218,1014.0849,"(20.0, 205.0)"
1,2.00,1948,1,5.95,True,MANUKA,19.108660,-155.825545,204.174455,"(20.0, 205.0)",...,29.034512,2.592494,-25.859348,0.589191,7.106411,2.945999,9.869999,23.385218,1014.0849,"(20.0, 205.0)"
2,2.20,1948,1,11.5,True,KAHUKU SHED 3,19.164740,-155.682280,204.317720,"(20.0, 205.0)",...,29.034512,2.592494,-25.859348,0.589191,7.106411,2.945999,9.869999,23.385218,1014.0849,"(20.0, 205.0)"
3,2.25,1948,1,5.515941,True,RESERVOIR (2940),19.160603,-155.822488,204.177512,"(20.0, 205.0)",...,29.034512,2.592494,-25.859348,0.589191,7.106411,2.945999,9.869999,23.385218,1014.0849,"(20.0, 205.0)"
4,2.26,1948,1,4.310617,True,CASTLE,19.225323,-155.778876,204.221124,"(20.0, 205.0)",...,29.034512,2.592494,-25.859348,0.589191,7.106411,2.945999,9.869999,23.385218,1014.0849,"(20.0, 205.0)"


In [621]:
dfs = []
for name, group in df_cdf_combined.groupby(
    by=['time']
):
    time = group['time'].unique()[0]
    # assert time.shape[0] == 1
    group = (
        group
        .sort_values(by=['lat', 'lon'])
        .drop(columns=['lat', 'lon', 'year', 'grid', 'month'])
        .melt(id_vars=['time'])
    )
    group['label'] = df.apply(lambda row: f"{row['variable']}_{row.name % 6}", axis=1)
    group = group.drop(columns=['time', 'variable']).transpose()
    group.columns = group.loc['label']
    group = group.drop('label')
    group['time'] = [time]
    dfs.append(
        group
    )

In [622]:
df_grid = pd.concat(dfs)
df_grid['year'] = df_grid['time'].dt.year
df_grid['month'] = df_grid['time'].dt.month

In [627]:
df_grid.head()

label,air2m_0,air2m_1,air2m_2,air2m_3,air2m_4,air2m_5,air1000_500_0,air1000_500_1,air1000_500_2,air1000_500_3,...,skt_5,slp_0,slp_1,slp_2,slp_3,slp_4,slp_5,time,year,month
value,295.726959,295.300873,295.396027,295.298279,294.962952,294.857971,31.169991,31.37999,31.299995,31.519989,...,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01,1948,1
value,295.328339,294.948212,294.804077,294.726898,294.499969,294.384399,31.490005,31.980003,32.330002,32.190002,...,22.042482,1015.343079,1015.235779,1015.476929,1016.235535,1015.979919,1015.875488,1948-02-01,1948,2
value,295.800781,295.274536,294.97821,295.364563,294.946594,294.533051,32.830002,32.670006,32.189995,32.87001,...,22.307774,1016.237976,1016.239624,1016.627136,1017.947021,1017.83136,1017.96936,1948-03-01,1948,3
value,296.61438,296.051941,295.796539,296.104492,295.702728,295.371246,30.32,30.140007,29.920006,30.399994,...,23.057003,1015.145142,1015.181641,1015.431641,1016.125305,1016.093994,1016.186157,1948-04-01,1948,4
value,297.482941,296.844818,296.500061,297.024323,296.539459,296.165039,29.990005,29.919991,29.710007,29.919998,...,24.052345,1016.440674,1016.371704,1016.578308,1017.76709,1017.747559,1017.908569,1948-05-01,1948,5


In [629]:
temp = pd.merge(left=df_w_closest_obs, right=df_grid, left_on=['year', 'month'], right_on=['year', 'month'])

In [630]:
temp

,skn,year,month,data_in,filled,name,lat,lon,lon_updated,closest_grid,...,skt_3,skt_4,skt_5,slp_0,slp_1,slp_2,slp_3,slp_4,slp_5,time_y
0,1.00,1948,1,3.2,False,KALAE,18.916176,-155.674994,204.325006,"(20.0, 205.0)",...,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01
1,2.00,1948,1,5.95,True,MANUKA,19.108660,-155.825545,204.174455,"(20.0, 205.0)",...,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01
2,2.20,1948,1,11.5,True,KAHUKU SHED 3,19.164740,-155.682280,204.317720,"(20.0, 205.0)",...,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01
3,2.25,1948,1,5.515941,True,RESERVOIR (2940),19.160603,-155.822488,204.177512,"(20.0, 205.0)",...,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01
4,2.26,1948,1,4.310617,True,CASTLE,19.225323,-155.778876,204.221124,"(20.0, 205.0)",...,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311,1948-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865532,1117.00,2011,12,6.42,False,PRINCEVILLE RCH,22.213196,-159.472201,200.527799,"(22.5, 200.0)",...,24.797201,24.190224,23.897985,1016.882812,1017.134338,1017.501221,1018.789612,1018.708801,1018.637451,2011-12-01
865533,1134.00,2011,12,4.73,False,KILAUEA,22.211941,-159.407202,200.592798,"(22.5, 200.0)",...,24.797201,24.190224,23.897985,1016.882812,1017.134338,1017.501221,1018.789612,1018.708801,1018.637451,2011-12-01
865534,1137.00,2011,12,7.15,False,KOLOKO RES,22.181408,-159.377758,200.622242,"(22.5, 200.0)",...,24.797201,24.190224,23.897985,1016.882812,1017.134338,1017.501221,1018.789612,1018.708801,1018.637451,2011-12-01
865535,1145.00,2011,12,2.44,False,PUU AUAU,22.182760,-159.332203,200.667797,"(22.5, 200.0)",...,24.797201,24.190224,23.897985,1016.882812,1017.134338,1017.501221,1018.789612,1018.708801,1018.637451,2011-12-01


In [578]:
df = dfs[0].melt(id_vars=['time'])
df['label'] = df.apply(lambda row: f"{row['variable']}_{row.name % 6}", axis=1)

In [579]:
temp = df.drop(columns=['time', 'variable']).transpose()
temp.columns = temp.loc['label']

In [580]:
t = temp.drop('label') 

In [581]:
assert df['time'].unique().shape[0] == 1
t.index = [df['time'].unique()]

In [582]:
t

label,air2m_0,air2m_1,air2m_2,air2m_3,air2m_4,air2m_5,air1000_500_0,air1000_500_1,air1000_500_2,air1000_500_3,...,skt_2,skt_3,skt_4,skt_5,slp_0,slp_1,slp_2,slp_3,slp_4,slp_5
1948-01-01,295.726959,295.300873,295.396027,295.298279,294.962952,294.857971,31.169991,31.37999,31.299995,31.519989,...,23.385218,23.6341,23.194658,23.052715,1014.175476,1013.94574,1014.0849,1015.194092,1014.699646,1014.406311


In [518]:
dfs[0].iloc[0].name

0

In [498]:
dfs = []
for name, group in df_cdf_combined.groupby(
    by=['time']
):
    # matrix.append(group)
    data = np.array(group.sort_values(by=['lat', 'lon']).drop(columns=['lat', 'lon', 'time', 'grid', 'year', 'month']))
    dfs.append(pd.DataFrame(data=[data], index=[group.iloc[0].time]))

ValueError: Must pass 2-d input. shape=(1, 6, 16)

In [492]:
for item in matrix:
    # data = np.array(matrix[0].sort_values(by=['lat', 'lon']).drop(columns=['lat', 'lon', 'time', 'grid', 'year', 'month']))
    data = np.array(item.sort_values(by=['lat', 'lon']).drop(columns=['lat', 'lon', 'time', 'grid', 'year', 'month']))
    # print(item.time.iloc[0])
    pd.DataFrame({"data": [data]}, index=[item.iloc[0].time])

,matrix
1948-01-01,"[[295.72696, 31.16999, 5800.355, 122.96774, 0...."
1948-02-01,"[[295.32834, 31.490005, 5794.3794, 132.27586, ..."
1948-03-01,"[[295.80078, 32.83, 5799.3228, 139.96774, -0.0..."
1948-04-01,"[[296.61438, 30.32, 5830.533, 131.9, 0.0179977..."
1948-05-01,"[[297.48294, 29.990005, 5859.5806, 142.7742, 0..."
...,...
2019-04-01,"[[297.50714, 31.085827, nan, nan, nan, -27.058..."
2019-05-01,"[[298.50052, 29.653221, nan, nan, nan, -29.050..."
2019-06-01,"[[299.5911, 31.349998, nan, nan, nan, -27.2227..."
2019-07-01,"[[300.21365, 31.38225, nan, nan, nan, -27.2971..."


In [303]:
df_ready_for_cdf.query('lon <= -156')

,skn,year,month,data_in,filled,name,lat,lon,lon_updated,closest_grid
77780,63.0,2006,1,0.363448,False,Kaloko-Honokohau,19.672778,-156.020278,203.979722,"[20.0, 205.0]"
77781,63.0,2007,1,0.86,False,Kaloko-Honokohau,19.672778,-156.020278,203.979722,"[20.0, 205.0]"
77782,63.0,2008,1,0.31,False,Kaloko-Honokohau,19.672778,-156.020278,203.979722,"[20.0, 205.0]"
77783,63.0,2009,1,1.28,False,Kaloko-Honokohau,19.672778,-156.020278,203.979722,"[20.0, 205.0]"
77784,63.0,2010,1,0.2,False,Kaloko-Honokohau,19.672778,-156.020278,203.979722,"[20.0, 205.0]"
...,...,...,...,...,...,...,...,...,...,...
1241277,425.5,2012,11,3.791339,False,HAIKU 3.2 SE HI US,20.876100,-156.304700,203.695300,"[20.0, 202.5]"
1241278,425.5,2012,12,6.913386,False,HAIKU 3.2 SE HI US,20.876100,-156.304700,203.695300,"[20.0, 202.5]"
1241279,830.6,1957,10,2.38,False,GAGE 4 MINK,21.466829,-157.933919,202.066081,"[22.5, 202.5]"
1241287,324.3,1949,12,1.5,False,KULA,20.740136,-156.317187,203.682813,"[20.0, 202.5]"
